# Analyzer Reporting Demo (Random Graph)

This notebook demonstrates the new analyzer/reporting workflow on a randomly generated factor graph. Each section highlights one of the core features added in the recent updates.


## 1. Imports and Setup

We bring in the PropFlow builders, the snapshot recorder, and the reporting utilities. All computations run on a small random graph so they execute quickly in the notebook environment.


In [ ]:
from pathlib import Path

import networkx as nx
import numpy as np

from propflow import BPEngine, FGBuilder
from propflow.configs import CTFactory
from analyzer.snapshot_recorder import EngineSnapshotRecorder
from analyzer.reporting import (
    SnapshotAnalyzer,
    AnalysisReport,
    parse_snapshots,
)


## 2. Build a Random Factor Graph and Capture Snapshots

We use `FGBuilder.build_random_graph` with the integer cost-table factory. The engine runs for a handful of iterations while the recorder captures every message exchange.


In [ ]:
# Construct a small random factor graph
np.random.seed(7)
random_fg = FGBuilder.build_random_graph(
    num_vars=6,
    domain_size=3,
    ct_factory=CTFactory.random_int.fn,
    ct_params={'low': 0, 'high': 8},
    density=0.6,
)

# Run a BP engine and capture per-step snapshots
engine = BPEngine(factor_graph=random_fg)
recorder = EngineSnapshotRecorder(engine)
raw_snapshots = recorder.record_run(max_steps=5)
len(raw_snapshots)


### Inspect the First Snapshot

The recorder returns plain dictionaries. We take a quick look at the message keys and sample values before parsing them into typed records.


In [ ]:
first_snapshot = raw_snapshots[0]
{
    'step': first_snapshot['step'],
    'message_count': len(first_snapshot['messages']),
    'assignments': first_snapshot['assignments'],
    'neutral_messages': first_snapshot['neutral_messages'],
}


## 3. Parse Snapshots and Register Factor Costs

`parse_snapshots` validates step ordering, argmin metadata, and neutral counts. We also gather the factor cost tables so neutrality checks and split-ratio recommendations can use the correct thresholds.


In [ ]:
records = parse_snapshots(raw_snapshots)
len(records)


In [ ]:
# Collect factor cost tables from the original graph
factor_tables = {
    factor.name: np.asarray(factor.cost_table, dtype=float)
    for factor in engine.factor_nodes
}

analyzer = SnapshotAnalyzer(records, max_cycle_len=6)
for name, table in factor_tables.items():
    analyzer.register_factor_cost(name, table)

factor_tables.keys()


## 4. Belief Trajectories per Variable

The analyzer reconstructs the argmin trajectory for each variable by aggregating factor-to-variable messages (mirroring the legacy visualizer logic).


In [ ]:
belief_series = analyzer.beliefs_per_variable()
belief_series


## 5. Difference Coordinates (ΔQ, ΔR)

We recenter variable→factor and factor→variable messages into difference coordinates. Binary domains collapse to scalar gaps; multi-label domains are shifted so their minima start at zero.


In [ ]:
delta_q, delta_r = analyzer.difference_coordinates(step_idx=0)
list(delta_q.items())[:3]


## 6. Jacobian Construction and Dependency Graph

The Jacobian is built in difference coordinates. Small systems use dense matrices automatically; the dependency digraph captures the non-zero pattern for graph-based diagnostics.


In [ ]:
J0 = analyzer.jacobian(step_idx=0)
J0_dense = J0.toarray() if hasattr(J0, 'toarray') else np.asarray(J0)
J0_dense


In [ ]:
dep_graph = analyzer.dependency_digraph(step_idx=0)
{
    'nodes': dep_graph.number_of_nodes(),
    'edges': dep_graph.number_of_edges(),
    'example_node': dep_graph.nodes[next(iter(dep_graph.nodes))],
}


## 7. Neutrality Checks and Greedy SCC Cover

We certify a sample factor step and then run the SCC-based greedy cover routine to break all directed cycles.


In [ ]:
record0 = records[0]
q_messages = {
    (msg.sender, msg.recipient): msg
    for msg in record0.messages
    if msg.flow == 'variable_to_factor'
}

sample_from_var, sample_factor = next(iter(q_messages.keys()))
# Pick a target variable receiving from the same factor
r_candidates = [
    msg.recipient
    for msg in record0.messages
    if msg.flow == 'factor_to_variable' and msg.sender == sample_factor
]

sample_to_var = r_candidates[0]
neutral_flag, winning_label = analyzer.neutral_step_test(
    step_idx=0,
    factor=sample_factor,
    from_var=sample_from_var,
    to_var=sample_to_var,
)
neutral_flag, winning_label


In [ ]:
cover, residual = analyzer.scc_greedy_neutral_cover(step_idx=0, alpha={})
cover, residual.number_of_nodes(), residual.number_of_edges()


## 8. Nilpotent Bounds, Block Norms, Cycles, and Split Ratios

The analyzer reports the nilpotent index (when it exists), the DAG longest-path bound, block norms consistent with the engine snapshot manager, cycle statistics, and heuristic split-ratio recommendations.


In [ ]:
nilpotent = analyzer.nilpotent_index(0)
dag_bound = analyzer._dag_bound_cache.get(0)
block_norms = analyzer.block_norms(0)
cycle_info = analyzer.cycle_metrics(0)
ratios = analyzer.recommend_split_ratios(0)
{
    'nilpotent_index': nilpotent,
    'dag_bound': dag_bound,
    'block_norms': block_norms,
    'cycle_metrics': cycle_info,
    'recommended_alpha': ratios,
}


## 9. Reporting Artefacts (JSON / CSV / Plots)

`AnalysisReport` bundles the analyzer results into reusable exports. The example below writes JSON and CSV summaries and produces belief/dependency plots under `results/analyzer_demo`.


In [ ]:
report = AnalysisReport(analyzer)
summary = report.to_json(step_idx=0)
summary


In [ ]:
output_dir = Path('results/analyzer_demo')
report.to_csv(output_dir, step_idx=0)
report.plots(output_dir, step_idx=0, include_graph=True)
sorted(p.name for p in output_dir.iterdir())


## 10. CLI Reference

Run the same analysis from the terminal with the dedicated entry point:

````
bp-analyze \
  --snapshots results/run.json \
  --out results/analyzer_demo \
  --step 0 \
  --compute-jac \
  --cover \
  --plot
````

This command mirrors the notebook flow: it parses snapshots, builds the analyzer, exports summaries, and optionally writes the Jacobian and neutral cover information.
